# Bitcoin Key Pairs

In [1]:
# pip install ecdsa

## Manual Key Generation

In [2]:
import ecdsa

# Generate a private key
private_key = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
print("Private key:", private_key.to_string().hex())

# Generate a public key
public_key = private_key.get_verifying_key()
print("Public key: ", public_key.to_string().hex())

Private key: 23a3d6df01d6c7904c9f7595dbf7fdab368e747ae4577549b4e53055d3c55a73
Public key:  47f0c9b313a10caf21f226d5316f17439d5cb98e26a47ec2579f9d1dddb21329019b610b97db7c38c39e29719acf56540c4cae0cf1ed693911d2a83e1ab15ede


While the public key is correct it is not in the proper format. 

## Bitcoin Address

Finding the bitcoin public key or bitcoin address from the private key is a bit more complicated.

The public key is a point on the elliptic curve, but it is not in the proper format. The proper format is a 65 byte array with the first byte being 0x04 and the next 32 bytes being the x coordinate and the last 32 bytes being the y coordinate. The x and y coordinates are 32 bytes each because they are 256 bit numbers. The first byte is 0x04 because it is an uncompressed public key. There are compressed public keys which are 33 bytes long, but we will not be using them.

### Create a bitcoin address

In [3]:
import hashlib
import base58

# Generate a bitcoin address
# Step 1: Hash public key
sha256 = hashlib.sha256(public_key.to_string()).digest()

# Step 2: Perform RIPEMD-160 hashing on the result of SHA256
ripemd160 = hashlib.new('ripemd160')
ripemd160.update(sha256)
hashed_public_key = ripemd160.digest()

# Step 3: Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
hashed_public_key = b'\x00' + hashed_public_key

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(hashed_public_key).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Add the 4 checksum bytes from stage 5 at the end of extended RIPEMD-160 hash from stage 4. This is the 25-byte binary Bitcoin Address.
binary_bitcoin_address = hashed_public_key + address_checksum

# Step 7: Convert the result from a byte string into a base58 string using Base58Check encoding. This is the most commonly used Bitcoin Address format
bitcoin_address = base58.b58encode(binary_bitcoin_address)
print("Bitcoin address: ", bitcoin_address.decode('utf-8'))

Bitcoin address:  1359Nv8usSywHX9dog9RHTgFrLtp5mzpRj


### Verify a bitcoin address

In [4]:
# Verify that the bitcoin address is correct
# Step 1: Decode Base58 Bitcoin Address
binary_bitcoin_address_check = base58.b58decode(bitcoin_address)

# Step 2: Extract Bitcoin Address from decoded Base58 Bitcoin Address
binary_bitcoin_address = binary_bitcoin_address_check[:-4]

# Step 3: Extract Bitcoin Address Checksum from decoded Base58 Bitcoin Address
binary_bitcoin_address_checksum = binary_bitcoin_address_check[-4:]

# Step 4: Perform SHA256 hash on the extended RIPEMD-160 result
double_sha256 = hashlib.sha256(hashlib.sha256(binary_bitcoin_address).digest()).digest()

# Step 5: Take the first 4 bytes of the second SHA256 hash. This is the address checksum
address_checksum = double_sha256[:4]

# Step 6: Verify that the address checksum from decoded Base58 Bitcoin Address is equal to the address checksum from the SHA256 hash
print("Bitcoin address is valid:", binary_bitcoin_address_checksum == address_checksum)

Bitcoin address is valid: True


## Signing Messages

### Sign a message

In [5]:
# Path: concepts_bitcoin_transactions.ipynb

# Use the private key to sign a message
message = b'Hello world'
signature = private_key.sign_deterministic(message)
print("Signature:", signature.__bytes__())
print("Signature:", signature.hex())


Signature: b'T\xf9j\xc9\xfb\x0f\x8d\xa12\x11f\x891*[{\x81<\xb1\x88\xb8\x11$\xdf,,\xac0\x9c:\xc9\x17\n\xef\xa3\xcc\xbd\x1bQ\xfb*C%4\xda,\xec[:#?\x92.g\xc4\x94g\xa3{\x06\xa1\xb4\xd3\xbf'
Signature: 54f96ac9fb0f8da132116689312a5b7b813cb188b81124df2c2cac309c3ac9170aefa3ccbd1b51fb2a432534da2cec5b3a233f922e67c49467a37b06a1b4d3bf


### Verify a message

In [6]:
# Use the public key to verify a message
print(public_key.verify(signature, message))

True


# Libraries for bitcoin keys

### pybitcoin

No longer maintained and does not work with python 3.12

In [7]:
# pip install pybitcoin

In [8]:
# from pybitcoin import BitcoinPrivateKey

# # Generate a private key
# private_key = BitcoinPrivateKey()
# print(f"Private key: {private_key.to_hex()}")

# # Generate a public key
# public_key = private_key.public_key()
# print(f"Public key: {public_key.to_hex()}")

# # Generate a bitcoin address
# bitcoin_address = public_key.address()
# print(f"Bitcoin address: {bitcoin_address}")

### bitcoin-utils

Package appears damaged and does not work with python 3.12

In [9]:
# pip install bitcoin-utils

In [10]:
# from bitcoinutils.keys import PrivateKey, PublicKey

# # Generate a private key
# private_key = PrivateKey()
# print(f"Private key: {private_key.to_wif(compressed=True)}")

# # get the public key
# public_key: PublicKey = private_key.get_public_key()
# print(f"Public key: {public_key.to_hex(compressed=True)}")

# # get the bitcoin address
# bitcoin_address = public_key.get_address()
# print(f"Bitcoin address: {bitcoin_address.to_string()}")
# print(f'Hash160: {bitcoin_address.to_hash160()}')



Sign a message with a private key and verify the signature with the public key.

In [11]:
# message: str = "Hello"
# signature = private_key.sign_message(message)
# print(f"Signature: {signature}")
# # print(f"Signature valid? {public_key.verify(message, signature)}")
# print(f"Signature valid? {public_key.verify_message(public_key.get_address().to_string(), message, signature)}")

### python-bitcoinlib

In [12]:
# pip install python-bitcoinlib

In [13]:
import secrets
import base64
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress
from bitcoin.signmessage import BitcoinMessage, VerifyMessage, SignMessage
from bitcoin.core import x, lx, b2x, b2lx, Hash160

message_: str = "Hello world!"

# Generate a private key
private_key_ = CBitcoinSecret.from_secret_bytes(secrets.token_bytes(32))
print(f"Private key: {private_key_}")

# Generate a public key
public_key_ = private_key_.pub
print(f"Public key: {public_key_.hex()}")

# Generate a bitcoin address
bitcoin_address_ = P2PKHBitcoinAddress.from_pubkey(public_key_.__bytes__())
print(f"Bitcoin address: {bitcoin_address_}")

# Sign message
bitcoin_msg = BitcoinMessage(message_)
signature_ = SignMessage(private_key_, bitcoin_msg)
print(f"Signature: {signature_.decode('utf-8')}")

# verify the message
print(f"Signature valid? {VerifyMessage(bitcoin_address_, bitcoin_msg, signature_)}")


Private key: KzYrCDja2F6q3RgUaUAJ3puuZZvNstMK58MHHs4cqyg4Q13xxhMP
Public key: 03a0bc05d71421eed49cdeb775b4e6387504e55a1f9d61d1fc6da36c617494ec69
Bitcoin address: 1JU7FHBVKVvscb12Q8xbFqmABPuABMBp6
Signature: H0qkcQaj5+Iu1nSVsJs6YJOrDetHLVW3kwU2OSQ0klFuBQF5yNCl+CnLn3U7YRH0LXm0QSEV/gJEGpL/UwcjHLM=
Signature valid? True


In [14]:

#verify the handmade message from the top
bitcoin_address_bytes = P2PKHBitcoinAddress.from_bytes(public_key.to_string())
print(f"Bitcoin address: {bitcoin_address_bytes}")

bitcoin_message_bytes = BitcoinMessage(message.decode('utf-8'))
print(f"Bitcoin message: {bitcoin_message_bytes.message}")

signature_bytes = base64.b64encode(bytes(b'\x00') + signature.__bytes__())
print(f"Signature: {signature_bytes.decode('utf-8')}")

print(f"Signature valid? {VerifyMessage(bitcoin_address_bytes, bitcoin_message_bytes, signature_bytes)}")

AttributeError: 'P2PKHBitcoinAddress' object has no attribute 'to_string'